In [1]:
import os
import ast
import pandas as pd
import numpy as np
from sklearn import model_selection
from tqdm import tqdm
import shutil

In [5]:
"""
The purpose of this python script is to create an unbiased training and validation set.
The split data will be run in the terminal calling a function (process_data) that will join the
annotations.csv file with new .txt files for bounding box class and coordinates for each image.
"""
# Credit to Abhishek Thakur, as this is a modified version of this notebook.
# Source to video, where he goes over his code: https://www.youtube.com/watch?v=NU9Xr_NYslo&t=1392s

# Import libraries
import os
import ast
import pandas as pd
import numpy as np
from sklearn import model_selection
from tqdm import tqdm
import shutil

# The DATA_PATH will be where your augmented images and annotations.csv files are.
# The OUTPUT_PATH is where the train and validation images and labels will go to.
DATA_PATH = 'D:/install/AI/Code/Project/yolo/V8/Yolov8/aug_data'
OUTPUT_PATH = 'D:/install/AI/Code/Project/yolo/V8/Yolov8/dataset'

def process_data(data, data_type='train'):
    # Tạo thư mục images và labels cho từng tập train, valid, test
    image_dir = os.path.join(OUTPUT_PATH, f"{data_type}/images")
    label_dir = os.path.join(OUTPUT_PATH, f"{data_type}/labels")
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    for _, row in tqdm(data.iterrows(), total=len(data)):
        image_name = row['image_id'][:-4]  # Bỏ phần mở rộng (ví dụ: .jpg)
        bounding_boxes = row['bboxes']
        yolo_data = []

        for bbox in bounding_boxes:
            category, x_center, y_center, w, h = bbox
            yolo_data.append([category, x_center, y_center, w, h])  # YOLO format

        yolo_data = np.array(yolo_data)

        # Lưu file label vào thư mục labels
        np.savetxt(
            os.path.join(label_dir, f"{image_name}.txt"),
            yolo_data,
            fmt=["%d", "%f", "%f", "%f", "%f"]
        )

        # Sao chép hình ảnh vào thư mục images
        shutil.copyfile(
            os.path.join(DATA_PATH, f"images/{image_name}.jpg"),
            os.path.join(image_dir, f"{image_name}.jpg"),
        )


if __name__ == '__main__':
    df = pd.read_csv(os.path.join(DATA_PATH, 'annotations.csv'))
    df.bbox = df.bbox.apply(ast.literal_eval) # Convert string to list for bounding boxes
    df = df.groupby('image_id')['bbox'].apply(list).reset_index(name='bboxes')
    # Run function to have our data ready for modeling in 03_Modeling_and_Inference.ipynb
    process_data(df, data_type='train')


100%|██████████| 967/967 [00:00<00:00, 1189.86it/s]
